Task:
- Extract data from 2025 for first data of the day
- Predict 2025 NPK level based on 2024 model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
'''import psycopg2
import pandas as pd

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="smart_composting_api",
    user="npds_2025",
    password="npds_2025_eh0atNiA5MVx2FYY3UnqVo5Vzv_0N9MRnSZ_3dkJgT_r2EIONEpFzV1o3IXHSFsjUX8hXT-9OgKqt8f512RPWJohKdM_pA-dAfimfXXOuke5C0Z9irOt4GrEV5R",
    host="db.composting.tinkerthings.global",
    port="6969"
)

# Define Custom SQL Query from PgAdmin

query = """
WITH RankedData AS (
  SELECT 
    SD.dataid, 
    D.devicename, 
    S.sensor, 
    S.sensorid, 
    SD.value, 
    DD.dbtimestamp,
    ROW_NUMBER() OVER (
      PARTITION BY 
        D.devicename, 
        S.sensorid, 
        DATE(DD.dbtimestamp)
      ORDER BY 
        DD.dbtimestamp ASC
    ) AS rn
  FROM public.sensordata AS SD
  JOIN public.devicedata   DD ON SD.dataid   = DD.dataid
  JOIN public.sensors      S  ON S.sensorid  = SD.sensorid
  JOIN public.devices      D  ON D.deviceid  = DD.deviceid
  WHERE D.devicename = 'NP002'
)
SELECT 
  dataid, 
  devicename, 
  sensor, 
  sensorid, 
  value, 
  dbtimestamp
FROM RankedData
WHERE rn = 1
ORDER BY dbtimestamp DESC;
"""

# Load result into DataFrame
chunks = pd.read_sql_query(query, conn, chunksize=1000)

# Combine or process each chunk
df_list = []
for chunk in chunks:
    print(f"Loaded chunk with {len(chunk)} rows.")
    df_list.append(chunk)  # Or process chunk directly

# Combine all chunks into a full DataFrame
df = pd.concat(df_list, ignore_index=True)
print(f"Total rows loaded: {len(df)}")

# Close connection
conn.close()'''

'import psycopg2\nimport pandas as pd\n\n# Connect to PostgreSQL\nconn = psycopg2.connect(\n    dbname="smart_composting_api",\n    user="npds_2025",\n    password="npds_2025_eh0atNiA5MVx2FYY3UnqVo5Vzv_0N9MRnSZ_3dkJgT_r2EIONEpFzV1o3IXHSFsjUX8hXT-9OgKqt8f512RPWJohKdM_pA-dAfimfXXOuke5C0Z9irOt4GrEV5R",\n    host="db.composting.tinkerthings.global",\n    port="6969"\n)\n\n# Define Custom SQL Query from PgAdmin\n\nquery = """\nWITH RankedData AS (\n  SELECT \n    SD.dataid, \n    D.devicename, \n    S.sensor, \n    S.sensorid, \n    SD.value, \n    DD.dbtimestamp,\n    ROW_NUMBER() OVER (\n      PARTITION BY \n        D.devicename, \n        S.sensorid, \n        DATE(DD.dbtimestamp)\n      ORDER BY \n        DD.dbtimestamp ASC\n    ) AS rn\n  FROM public.sensordata AS SD\n  JOIN public.devicedata   DD ON SD.dataid   = DD.dataid\n  JOIN public.sensors      S  ON S.sensorid  = SD.sensorid\n  JOIN public.devices      D  ON D.deviceid  = DD.deviceid\n  WHERE D.devicename = \'NP002\'\n)\nSELECT

In [3]:
# Double Check Data
# Will be using real-time data to check on compost status

'''df.to_csv("2025 Raw Data.csv ", index=False)'''

'df.to_csv("2025 Raw Data.csv ", index=False)'

In [4]:
# load data
df = pd.read_csv('2025 Raw Data.csv')

In [5]:
df.head()

,dataid,devicename,sensor,sensorid,value,dbtimestamp
0,71267360,NP002,Soil Moisture,9,61.0,2025-05-28 00:00:24.345440
1,71267365,NP002,Soil Phosphorus,13,824.0,2025-05-28 00:00:24.345440
2,71267361,NP002,Soil Temperature,8,29.9,2025-05-28 00:00:24.345440
3,71267362,NP002,Soil EC,11,1798.0,2025-05-28 00:00:24.345440
4,71267364,NP002,Soil Nitrogen,12,336.0,2025-05-28 00:00:24.345440


In [6]:
df['dbtimestamp'] = df['dbtimestamp'].astype('datetime64[ns]')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dataid       150 non-null    int64         
 1   devicename   150 non-null    object        
 2   sensor       150 non-null    object        
 3   sensorid     150 non-null    int64         
 4   value        150 non-null    float64       
 5   dbtimestamp  150 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 7.2+ KB


Feature Engineering: Number of Days since First Recording

In [8]:
df['days_from_start'] = (df['dbtimestamp'] - df['dbtimestamp'].min()).dt.days

In [9]:
df.head()

,dataid,devicename,sensor,sensorid,value,dbtimestamp,days_from_start
0,71267360,NP002,Soil Moisture,9,61.0,2025-05-28 00:00:24.345440,13
1,71267365,NP002,Soil Phosphorus,13,824.0,2025-05-28 00:00:24.345440,13
2,71267361,NP002,Soil Temperature,8,29.9,2025-05-28 00:00:24.345440,13
3,71267362,NP002,Soil EC,11,1798.0,2025-05-28 00:00:24.345440,13
4,71267364,NP002,Soil Nitrogen,12,336.0,2025-05-28 00:00:24.345440,13


Format dataframe so each sensor becomes column and value is the cell content

In [10]:
# Convert dbtimestamp to get day | Since we are extracting one data a day now, only the date matters
# Once we start extracting more, I will create a timebucket to the nearest 10 seconds so that readings that happen within the same bucket groups together

df['date'] = df['dbtimestamp'].dt.floor('D')   

In [11]:
df.head()

,dataid,devicename,sensor,sensorid,value,dbtimestamp,days_from_start,date
0,71267360,NP002,Soil Moisture,9,61.0,2025-05-28 00:00:24.345440,13,2025-05-28
1,71267365,NP002,Soil Phosphorus,13,824.0,2025-05-28 00:00:24.345440,13,2025-05-28
2,71267361,NP002,Soil Temperature,8,29.9,2025-05-28 00:00:24.345440,13,2025-05-28
3,71267362,NP002,Soil EC,11,1798.0,2025-05-28 00:00:24.345440,13,2025-05-28
4,71267364,NP002,Soil Nitrogen,12,336.0,2025-05-28 00:00:24.345440,13,2025-05-28


In [12]:
# Pivot with both date and devicename in the index
npk_row = df.pivot(index=['date', 'devicename'], columns='sensor', values='value').reset_index()

# Sort by dbtimestamp
npk_row = npk_row.sort_values(by='date', ascending=False).reset_index(drop=True)

# Display result
npk_row.head()

sensor,date,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,2025-05-28,NP002,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0
1,2025-05-27,NP002,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1
2,2025-05-26,NP002,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9
3,2025-05-25,NP002,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9
4,2025-05-24,NP002,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0


Categorize into Numerical Columns

In [13]:
import numpy as np
import pandas as pd

num_cols = [
    n for n in npk_row.columns
    if npk_row[n].dtypes != 'O' and not np.issubdtype(npk_row[n].dtype, np.datetime64)
]

df_num = npk_row[num_cols]

df_num.head()

sensor,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH
0,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0
1,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1
2,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9
3,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9
4,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0


Null Imputation: Not Needed

In [14]:
print(npk_row.isnull().mean().sort_values(ascending = False))

sensor
date                0.0
devicename          0.0
Air Temperature     0.0
CO2                 0.0
Humidity            0.0
Soil EC             0.0
Soil Moisture       0.0
Soil Nitrogen       0.0
Soil Phosphorus     0.0
Soil Potassium      0.0
Soil Temperature    0.0
Soil pH             0.0
dtype: float64


Create Calculation Method for NPK:

*Will have to review non-used columns and method*

In [15]:
def calculate_npk_ratio(row):
    N = row['Soil Nitrogen']
    P = row['Soil Phosphorus']
    K = row['Soil Potassium']

    # Avoid division by zero
    if min(N, P, K) == 0:
        return float('nan')

    # Normalize based on smallest value
    min_val = min(N, P, K)
    return f"{round(N/min_val, 1)} : {round(P/min_val, 1)} : {round(K/min_val, 1)}"

In [16]:
# Create new column
npk_row['NPK Ratio'] = npk_row.apply(calculate_npk_ratio, axis=1)

# View result
npk_row.head()

sensor,date,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio
0,2025-05-28,NP002,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0,1.0 : 2.5 : 2.4
1,2025-05-27,NP002,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1,1.0 : 2.4 : 2.4
2,2025-05-26,NP002,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9,1.0 : 2.6 : 2.5
3,2025-05-25,NP002,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9,1.0 : 2.6 : 2.6
4,2025-05-24,NP002,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0,1.0 : 2.6 : 2.6


Categorize into 'Low', 'High', 'Optimal'
- Long Bean: 1:2:2
- Spinach: 2:1:1
- Lady Finger 1:1:1

In [17]:
def nitrogen(value):
    if value <= 100:
        return 'Low'
    elif 100 < value <= 300:
        return 'High'
    else:
        return 'Optimal'

In [18]:
def pp(value):
    if value <= 300:
        return 'Low'
    elif 300 < value <= 600:
        return 'High'
    else:
        return 'Optimal'

In [19]:
npk_row['N Category'] = npk_row['Soil Nitrogen'].apply(nitrogen)
npk_row['P Category'] = npk_row['Soil Phosphorus'].apply(pp)
npk_row['K Category'] = npk_row['Soil Potassium'].apply(pp)

# View result
npk_row.head()

sensor,date,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category
0,2025-05-28,NP002,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0,1.0 : 2.5 : 2.4,Optimal,Optimal,Optimal
1,2025-05-27,NP002,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1,1.0 : 2.4 : 2.4,Optimal,Optimal,Optimal
2,2025-05-26,NP002,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9,1.0 : 2.6 : 2.5,High,High,High
3,2025-05-25,NP002,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9,1.0 : 2.6 : 2.6,High,High,High
4,2025-05-24,NP002,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0,1.0 : 2.6 : 2.6,High,High,High


In [20]:
# Long Bean: 1:2:2
# Spinach: 2:1:1
# Lady Finger 1:1:1

def categorize_npk(row):
    ratio = row['NPK Ratio']

    if ratio == '1.0 : 2.0 : 2.0':
        return 'Long Bean'
    elif ratio == '2.0 : 1.0 : 1.0':
        return 'Spinach'
    elif ratio == '1.0 : 1.0 : 1.0':
        return 'Lady Finger'
    else:
        return 'Invalid'

In [21]:
npk_row['NPK Level'] = npk_row.apply(categorize_npk, axis=1)

# View result
npk_row.head()

sensor,date,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level
0,2025-05-28,NP002,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0,1.0 : 2.5 : 2.4,Optimal,Optimal,Optimal,Invalid
1,2025-05-27,NP002,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1,1.0 : 2.4 : 2.4,Optimal,Optimal,Optimal,Invalid
2,2025-05-26,NP002,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9,1.0 : 2.6 : 2.5,High,High,High,Invalid
3,2025-05-25,NP002,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9,1.0 : 2.6 : 2.6,High,High,High,Invalid
4,2025-05-24,NP002,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0,1.0 : 2.6 : 2.6,High,High,High,Invalid


In [22]:
def npk_done(row):
    ratio = row['NPK Ratio']
    n = row['N Category']
    p = row['P Category']
    k = row['K Category']

    if ratio == '1.0 : 2.0 : 2.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Long Bean Done'
    elif ratio == '2.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Spinach Done'
    elif ratio == '1.0 : 1.0 : 1.0' and n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Lady Finger Done'
    elif n == 'Optimal' and p == 'Optimal' and k=='Optimal':
        return 'Optimal NPK'
    else:
        return 'Not Done Composting'

In [23]:
npk_row['NPK Done'] = npk_row.apply(npk_done, axis=1)

# View result
npk_row.head()

sensor,date,devicename,Air Temperature,CO2,Humidity,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-28,NP002,27.937360,5270.0,100.0,1798.0,61.0,336.0,824.0,822.0,29.9,5.0,1.0 : 2.5 : 2.4,Optimal,Optimal,Optimal,Invalid,Optimal NPK
1,2025-05-27,NP002,28.826582,7162.0,100.0,1840.0,59.4,345.0,844.0,843.0,30.3,5.1,1.0 : 2.4 : 2.4,Optimal,Optimal,Optimal,Invalid,Optimal NPK
2,2025-05-26,NP002,27.542152,1041.0,100.0,1059.0,27.4,182.0,467.0,463.0,28.3,4.9,1.0 : 2.6 : 2.5,High,High,High,Invalid,Not Done Composting
3,2025-05-25,NP002,27.739758,1080.0,100.0,997.0,26.6,169.0,438.0,433.0,28.3,4.9,1.0 : 2.6 : 2.6,High,High,High,Invalid,Not Done Composting
4,2025-05-24,NP002,27.566187,1202.0,100.0,938.0,25.5,157.0,409.0,404.0,28.2,5.0,1.0 : 2.6 : 2.6,High,High,High,Invalid,Not Done Composting


In [24]:
extract = npk_row.copy()

In [25]:
extract.count()

sensor
date                15
devicename          15
Air Temperature     15
CO2                 15
Humidity            15
Soil EC             15
Soil Moisture       15
Soil Nitrogen       15
Soil Phosphorus     15
Soil Potassium      15
Soil Temperature    15
Soil pH             15
NPK Ratio           15
N Category          15
P Category          15
K Category          15
NPK Level           15
NPK Done            15
dtype: int64

Remove unneccesary columns to match 2024

In [26]:
# List of columns to exclude
exclude_cols = ['Air Temperature', 'CO2', 'Humidity']

# Select all columns except the excluded ones
df_filtered = extract.drop(columns=exclude_cols)

df_filtered.head()

sensor,date,devicename,Soil EC,Soil Moisture,Soil Nitrogen,Soil Phosphorus,Soil Potassium,Soil Temperature,Soil pH,NPK Ratio,N Category,P Category,K Category,NPK Level,NPK Done
0,2025-05-28,NP002,1798.0,61.0,336.0,824.0,822.0,29.9,5.0,1.0 : 2.5 : 2.4,Optimal,Optimal,Optimal,Invalid,Optimal NPK
1,2025-05-27,NP002,1840.0,59.4,345.0,844.0,843.0,30.3,5.1,1.0 : 2.4 : 2.4,Optimal,Optimal,Optimal,Invalid,Optimal NPK
2,2025-05-26,NP002,1059.0,27.4,182.0,467.0,463.0,28.3,4.9,1.0 : 2.6 : 2.5,High,High,High,Invalid,Not Done Composting
3,2025-05-25,NP002,997.0,26.6,169.0,438.0,433.0,28.3,4.9,1.0 : 2.6 : 2.6,High,High,High,Invalid,Not Done Composting
4,2025-05-24,NP002,938.0,25.5,157.0,409.0,404.0,28.2,5.0,1.0 : 2.6 : 2.6,High,High,High,Invalid,Not Done Composting


In [27]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              15 non-null     datetime64[ns]
 1   devicename        15 non-null     object        
 2   Soil EC           15 non-null     float64       
 3   Soil Moisture     15 non-null     float64       
 4   Soil Nitrogen     15 non-null     float64       
 5   Soil Phosphorus   15 non-null     float64       
 6   Soil Potassium    15 non-null     float64       
 7   Soil Temperature  15 non-null     float64       
 8   Soil pH           15 non-null     float64       
 9   NPK Ratio         15 non-null     object        
 10  N Category        15 non-null     object        
 11  P Category        15 non-null     object        
 12  K Category        15 non-null     object        
 13  NPK Level         15 non-null     object        
 14  NPK Done          15 non-nul

Extract Data

In [28]:
df_filtered.to_csv("2025 Extracted.csv ", index=False)